In [1]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
from util.plots import scatter_plot
from typing import List, Dict
from tqdm.notebook import tqdm

<IPython.core.display.Javascript object>

In [3]:
np.random.seed(seed=42)

CELL_DIMENSION = np.array([16, 16])
orientation = 60
scale = 10
rotation_matrix = np.array(
    [
        [math.cos(orientation), -math.sin(orientation)],
        [math.sin(orientation), math.cos(orientation)],
    ]
)

PHASES_PER_UNIT_DISTANCE = 1.0 / np.array([scale, scale])

<IPython.core.display.Javascript object>

In [6]:
np.random.seed(seed=42)

orientation = 60
scale = 10
rotation_matrix = np.array(
    [
        [math.cos(orientation), -math.sin(orientation)],
        [math.sin(orientation), math.cos(orientation)],
    ]
)

PHASES_PER_UNIT_DISTANCE = 1.0 / np.array([scale, scale])

<IPython.core.display.Javascript object>

In [7]:
NO_COLUMNS = 32
CELLS_PER_COLUMN = 16
sensory_layer_activity = np.zeros((NO_COLUMNS, CELLS_PER_COLUMN))  # A_t^in

<IPython.core.display.Javascript object>

In [8]:
NO_GRID_CELL_MODULES = 32
NO_GRID_CELLS_PER_AXIS = 16
GRID_MODULE_DIM = np.array([16, 16])
TOTAL_GRID_CELLS_PER_MOD = np.prod(GRID_MODULE_DIM)
location_layer_activity = np.zeros(
    (NO_GRID_CELL_MODULES, TOTAL_GRID_CELLS_PER_MOD, NO_COLUMNS, CELLS_PER_COLUMN)
)  # A_t^loc
# activation states:
# sense
# loc

<IPython.core.display.Javascript object>

In [9]:
dendrites_to_sense_layer = np.zeros(
    (NO_GRID_CELL_MODULES, TOTAL_GRID_CELLS_PER_MOD, NO_COLUMNS, CELLS_PER_COLUMN)
)  # D_c,d^in
dendrites_to_loc_layer = np.zeros((NO_COLUMNS, CELLS_PER_COLUMN))  # D_c,d^loc

# vector for active dendritic segments
# pi_t^in
# pi_t^loc
# dendritic threshold:
# sig^in
# sig^loc

<IPython.core.display.Javascript object>

### Algorithm

1. movement to update location layer
    - calculate new phase - input: displacement vector and old phase; applies translation by rotation and scaling + mod 1 --> outputs binary vector A_t,move^loc
    - if no movement is given, activate random location ???
2. predict sensory information from location layer output
    - predictions represented by pi_t^in = activity of distal dendritic segments - Eq2 in paper
        --> modularoty effect
3. calculate activity in sensory layer
    - all cells share same feedforward receptive fields --> pretrained CNN ouputs (128,) feature vector and each column is mapped to on feature
    - sensory features represented by sparse subset of mini-columns = W_t^in
    - if cell is predicted and in active mini-columns: activate cell and inhibit other cells in mini-column; else: all cells become active
    - - multiple cells can be active if predicted by location layer
    - formula uq: 6 for activation logic
4. update location layer based on sensory cues
    - location layer receives input from sensory layer (in form of W_t^in???)
    - calculate overlap EQ 3 & 4
    - repeat because new movement is received

Learning:
- strenghtening between connections of both layer
- when new object is learned: location representation is randomly initialised (each object has its own location space = multiple objects can be represented at the same time because their representation has low probability of overlap
- sensory layer: random cell in each column will be active
- form reciprocal connections on one of their dendritic segments (d') (EQ 5 & 6)
--> if synapse exist it is unaffected otherwise it is formed

In [10]:
THRESHOLD = 10

<IPython.core.display.Javascript object>

In [17]:
active_cells = compute_active_cells(get_random_phases())
active_cells

array([223, 103,  94, 229, 192,  18, 241,  30,  45, 101, 194,  15, 148,
       162,  19, 228, 128, 123, 205,  75, 162, 161,  18, 139, 229, 196,
       169,  15,  53,  56, 164, 110])

<IPython.core.display.Javascript object>

In [68]:
def get_random_phases():
    active_phases = np.random.random((NO_GRID_CELL_MODULES, 2))

    return active_phases


def compute_active_cells(active_phases):
    active_cell_coordinates = np.floor(active_phases * CELL_DIMENSION).astype(int)

    cells_for_active_phases = np.ravel_multi_index(
        active_cell_coordinates.T, CELL_DIMENSION
    )

    return cells_for_active_phases

def compute_movement(displacement: np.ndarray, active_phases):
    phase_displacement = (
        np.matmul(rotation_matrix, displacement) * PHASES_PER_UNIT_DISTANCE
    )

    active_phases = active_phases + phase_displacement

    active_phases = np.round(active_phases, decimals=9)
    active_phases = np.mod(active_phases, 1.0)

    return active_phases

def learn_object(obj: List[Dict], learn=False):
    """
    obj: [{location: <displacement vector>, feature: <feature_vector>}, ...]

    """

    # generate random phases since location is not given before first iteration
    phases = [get_random_phases()]

    for sample in obj:

        location = sample["location"]  # shape: (<2>) -> displacement vector
        feature = sample["feature"]  # shape: (<no_columns>)
        
        # get current active_phase: either random phase 
        # or calculate as movement from displacement and previois phase
        active_phase = phases[-1]
        if location:
            active_phase = compute_movement(location, phases[-1])
            phases.append(active_phase)
            
        
        active_grid_cell_indices = compute_active_cells(active_phase)

        # get overlap of phases and dendritic distal segments to predict cells and
        # check if number of dendritic segments are above threshold for cell to be predicted
        predicted_cells = np.sum(
            dendrites_to_sense_layer[:, active_cells], axis=(0, 1)
        )  # shape: (<no_columns>, <no_cells>)
        predicted_cells[predicted_cells < THRESHOLD] = 0

        # get active columns from input feature
        active_columns = np.argwhere(feature)

        # set predicted cells in inactive columns to zero
        predicted_cells[~active_columms] = 0

        # choose active cells:
        # if active column has predicted cells, set cells to active -> copied from predicted_cells
        # if not either set all cells to be active or during learning choose one random cell
        active_cells = np.copy(predicted_cells)
        active_cells[active_cells > 0] = 1

        if learning:
            no_pred_cells_per_column = np.sum(
                predicted_cells, axis=1
            )  # shape: (<no_columns>,)
            predicted_cells[active_columns & (no_pred_cells_per_column == 0)] = 1
        else:
            no_columns_to_choose_cell = predicted_cells[
                active_columns & (no_pred_cells_per_column == 0)
            ].shape[
                0
            ]  # number of columns to chose random cell for
            
            active_random_cells = np.random.randint(0, CELLS_PER_COLUMN + 1, size=(no_columns_to_choose_cell))
            
            active_cells[no_columns_to_choose_cell, active_random_cells] = 1
            
            

        # update location representation by calculating overlap whereas on active dendritic segment is enough

        # form reciprocal connection  of dendritic segments on overlapping cells


<IPython.core.display.Javascript object>